In [2]:
from scipy.sparse.linalg import svds
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#test data 만들기 #######################

In [10]:
#user1~30까지 모든 제품에 대한 평점 랜덤으로 생성
ratings={'userid':[i for i in range(1,31) for j in range(1, 5902) ],
    'productid':[i for j in range(1, 31) for i in range(1,5902)],
     'rating':[np.random.randint(1,6) for i in range(1,177031)]
      }

ratings = pd.DataFrame(ratings)
ratings.head(4)

,userid,productid,rating
0,1,1,5
1,1,2,2
2,1,3,3
3,1,4,2


In [12]:
# 평점 데이터 저장
ratings.to_excel('./datasets/ratings.xlsx', index = True)

In [9]:
# 저장한 학습data에서 testdata 추출
ratings = pd.read_excel("./datasets/ratings.xlsx")

In [10]:
# ratings데이터에서 랜덤으로 테스트 데이터 10% 추출 
index = [np.random.randint(0,177030) for i in range(0,53109)]
ratings = ratings.iloc[index]

In [11]:
ratings = ratings.reset_index(drop=True)
ratings.head(4)

,userid,productid,rating
0,12,2410,1
1,8,4564,1
2,6,214,4
3,25,2845,5


In [12]:
ratings.to_excel('./datasets/test_data3.xlsx', index = True)

In [ ]:
#복합기 크롤링 데이터 수정###########

In [43]:
data = pd.read_excel("./datasets/복합기 크롤링 중복제거.xlsx")
data=data.fillna('')

In [44]:
#나누어져 있는 스펙 목록 컬럼을 keyword로 합침
cols = ['Unnamed: ' + str(i) for i in range(3,30)]
data['keyword'] = data[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
colss=['스펙 목록', 'keyword']
data['keyword'] = data[colss].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

In [45]:
# 상품명, 가격, 스펙목록, keyword 남기고 컬럼 제거
data = data.drop(cols, axis=1)
data = data.drop('스펙 목록', axis=1)

In [46]:
# 상품명 번호(productid) 컬럼 생성
data['productid'] = [i for i in range(1,5902)]
data.head(4)

,상품명,가격,keyword,productid
0,ApeosPort 2560,2125000,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...,1
1,ApeosPort 3060,2200000,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...,2
2,ApeosPort 3560,2528000,"흑백 디지털 복합기 잉크젯 컬러 출력 복사 스캔 인쇄 해상도: 4,800d...",3
3,ApeosPort C2060,2600000,컬러 디지털 복합기 인쇄 복사 스캔 컬러 속도: 23ppm 흑백 속도: 2...,4


In [47]:
data.to_excel('./datasets/printer_data.xlsx', index = False)

In [ ]:
# 사용자 data추가 ###############################

In [26]:
job = ['법','의류','교육','서비스','사무','건설','은행','병원','행정','기타']
user={'userid':[i for i in range(1,31)],
     '업종':[job[np.random.randint(0,10)] for i in range(1,31)],
     '용지사용량':[np.random.randint(100,3001) for i in range(1,31)],
      '내구성': [np.random.randint(1,5) for i in range(1,31)],
      '근무자 수' : [np.random.randint(1,101) for i in range(1,31)],
      '가격' : [np.random.randint(0,20)*1000000 for i in range(1,31)]
      }
user = pd.DataFrame(user)
user.head(4)

,userid,업종,용지사용량,내구성,근무자 수,가격
0,1,건설,1503,1,33,14000000
1,2,병원,2036,1,41,7000000
2,3,법,123,1,85,14000000
3,4,병원,253,3,43,15000000


In [27]:
user.to_excel('./datasets/user_data.xlsx', index = False)

In [56]:
# 복합기 데이터에 용지 사용량 추가 #####################
paper=''
for i in range(0, 5901):
    keyword = data.loc[i]['keyword']
    if keyword.find('기본 급지용량') == -1:
        paper[i]='-1'
    else:
        keyword_extract = keyword[keyword.find('기본 급지용량'):keyword.find('기본 급지용량')+15]
        paper[i]= keyword_extract

TypeError: 'str' object does not support item assignment

In [14]:
datata = pd.read_excel("./datasets/printer_data.xlsx")
datata.head()

,상품명,가격,keyword,productid
0,ApeosPort 2560,2125000,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...,1
1,ApeosPort 3060,2200000,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...,2
2,ApeosPort 3560,2528000,"흑백 디지털 복합기 잉크젯 컬러 출력 복사 스캔 인쇄 해상도: 4,800d...",3
3,ApeosPort C2060,2600000,컬러 디지털 복합기 인쇄 복사 스캔 컬러 속도: 23ppm 흑백 속도: 2...,4
4,ApeosPort C2560,2778170,컬러 디지털 복합기 인쇄 복사 스캔 팩스 컬러 속도: 23ppm 흑백 속...,5


In [15]:
#print(datata.loc[3]['keyword'])
datata = pd.DataFrame(datata)
datata['기본 급지용량'] = str(-1)
aa = '기본 급지용량:'

#print(datata.loc[2]['기본 급지용량'])
#datata.at[2,'기본 급지용량'] = 99
#print(datata.loc[2]['기본 급지용량'])


for i in range(0,5901):

    if( aa in datata.loc[i]['keyword']):
        start = datata.loc[i]['keyword'].find(aa)
        #print(datata.loc[i]['기본 급지용량'])
        datata.at[i,'기본 급지용량'] = datata.loc[i]['keyword'][start+9:start+15]
        #print(datata.loc[i]['기본 급지용량'])

datata

,상품명,가격,keyword,productid,기본 급지용량
0,ApeosPort 2560,2125000,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...,1,-1
1,ApeosPort 3060,2200000,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...,2,-1
2,ApeosPort 3560,2528000,"흑백 디지털 복합기 잉크젯 컬러 출력 복사 스캔 인쇄 해상도: 4,800d...",3,-1
3,ApeosPort C2060,2600000,컬러 디지털 복합기 인쇄 복사 스캔 컬러 속도: 23ppm 흑백 속도: 2...,4,"1,200매"
4,ApeosPort C2560,2778170,컬러 디지털 복합기 인쇄 복사 스캔 팩스 컬러 속도: 23ppm 흑백 속...,5,"1,350매"
...,...,...,...,...,...
5896,코니카미놀타 bizhub C451,0,컬러 디지털 복합기 레이저 컬러 출력 복사 스캔 컬러 속도: 21ppm ...,5897,-1
5897,코니카미놀타 C022DN,0,컬러 디지털 복합기 잉크젯 컬러 출력 복사 스캔 컬러 속도(ISO): 6....,5898,-1
5898,코니카미놀타 C028DN,0,흑백 디지털 복합기 레이저젯 인쇄 컬러 속도: 38ppm 흑백 속도: 38p...,5899,650매
5899,코니카미놀타 C036DN,0,흑백 디지털 복합기 레이저 컬러 출력 복사 스캔 팩스 컬러 속도: 4pp...,5900,-1


In [ ]:
## test ##

In [28]:
data = pd.read_excel("./datasets/printer_data.xlsx")
data2 = pd.read_excel("./datasets/user_data.xlsx")

In [42]:
vector = CountVectorizer(ngram_range=(1,3))
vector_keyword = vector.fit_transform(data['업종'])
vector_keyword2 = vector.fit_transform(data2['업종'])

In [37]:
keyword_c_sim = cosine_similarity(vector_keyword, vector_keyword2).argsort()[:, ::-1]
keyword_c_sim

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 9 while Y.shape[1] == 7

In [36]:
keyword_c_sim.shape

(5901, 5901)

In [40]:
cosine_similarity(123, 0).argsort()[:, ::-1]

ValueError: Expected 2D array, got scalar array instead:
array=123.0.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.